0. 분석 대상 종목 list → df_tickers.csv
1. 기본적 분석 데이터 30일 → df_tickers.xlsx
2. 기술적 분석 데이터 365일 → volatility_df.xlsx
3. merge 및 엑셀 수정 → merged_df.xlsx

# 1. 기본적 분석 데이터 30일

In [14]:
from pykrx import stock
import pandas as pd
import datetime
from tqdm import tqdm
import time
import numpy as np

# 안전한 최근 영업일 계산 함수
def get_safe_nearest_business_day():
    """가장 가까운 영업일을 안전하게 가져오는 함수"""
    try:
        return stock.get_nearest_business_day_in_a_week()
    except IndexError:
        # 만약 영업일 데이터가 없을 경우, 현재 날짜를 반환
        return datetime.datetime.today().strftime('%Y%m%d')

# 최근 30일 거래 데이터로 기본적 분석하기.
days_ago = 30
sleep_time = 0.2  # API 호출 간 대기 시간 (초)

# 오늘 날짜 계산
today = get_safe_nearest_business_day()

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# 종목명 기존 파일에서 가져오기.
dfticker = pd.read_csv('df_tickers.csv', encoding='cp949')
tickers = dfticker[['Ticker', 'Market']]
df_tickers = pd.DataFrame(tickers, columns=['Ticker', 'Market'])

# Ticker_name 열 추가 (종목명)
df_tickers['Ticker_name'] = df_tickers['Ticker'].apply(lambda x: stock.get_market_ticker_name(x))

# sample로 50개만 돌리기
#df_tickers = df_tickers.head(50)

# Error handling 및 API Rate Limits 추가
def safe_api_call(api_func, ticker, *args, fallback_value=np.nan, desc=""):
    """안전한 API 호출을 위한 함수"""
    try:
        result = api_func(*args, ticker)
        time.sleep(sleep_time)  # API 호출 간 대기 시간
        return result
    except Exception as e:
        print(f"Error fetching {desc} for {ticker}: {e}")
        return fallback_value

# 시가총액 데이터를 한 번의 호출로 가져옴
def add_market_cap_data(df_tickers, start_date, end_date):
    for ticker in tqdm(df_tickers['Ticker'], desc="Adding Market Cap Data"):
        df = safe_api_call(stock.get_market_cap_by_date, ticker, start_date, end_date, desc="Market Cap Data")
        market_cap = df['시가총액'].iloc[-1] if not df.empty else np.nan
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'Market_Cap'] = market_cap
    return df_tickers

# OHLCV 데이터를 한 번의 호출로 가져옴
def add_ohlcv_data(df_tickers, start_date, end_date):
    for ticker in tqdm(df_tickers['Ticker'], desc="Adding OHLCV Data"):
        df = safe_api_call(stock.get_market_ohlcv_by_date, ticker, start_date, end_date, desc="OHLCV Data")
        avg_volume_30d = round(df['거래량'].mean(), 2) if not df.empty else np.nan
        trading_value = round((df['종가'] * df['거래량']).mean(), 2) if not df.empty else np.nan  # 거래금액 계산
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'Avg_Volume_30D'] = avg_volume_30d
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'Avg_Trading_Value'] = trading_value  # 거래금액 추가
    return df_tickers

# Fundamental 데이터를 한 번의 호출로 가져옴
def add_fundamental_data(df_tickers, start_date, end_date):
    for ticker in tqdm(df_tickers['Ticker'], desc="Adding Fundamental Data"):
        df = safe_api_call(stock.get_market_fundamental_by_date, ticker, start_date, end_date, desc="Fundamental Data")
        eps = df['EPS'].iloc[-1] if not df.empty and pd.notna(df['EPS'].iloc[-1]) else np.nan
        per = df['PER'].iloc[-1] if not df.empty and pd.notna(df['PER'].iloc[-1]) else np.nan
        pbr = df['PBR'].iloc[-1] if not df.empty and pd.notna(df['PBR'].iloc[-1]) else np.nan
        dividend_yield = df['DIV'].iloc[-1] if not df.empty and pd.notna(df['DIV'].iloc[-1]) else np.nan
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'PER'] = per
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'PBR'] = pbr
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'EPS'] = eps
        df_tickers.loc[df_tickers['Ticker'] == ticker, 'Dividend_Yield'] = dividend_yield
    return df_tickers

# 모든 데이터를 한 번에 처리
# 기존 코드의 Market Cap 부분을 이 함수로 대체
df_tickers = add_ohlcv_data(df_tickers, start_date, today)
df_tickers = add_fundamental_data(df_tickers, start_date, today)
df_tickers = add_market_cap_data(df_tickers, start_date, today)
df_tickers['Ticker'] = df_tickers['Ticker'].astype(str).str.zfill(6)

# 엑셀 파일로 저장
df_tickers.to_excel('df_tickers.xlsx', index=False)

# 상위 5개 출력
df_tickers.head()


Adding Fundamental Data:  26%|██▋       | 714/2712 [03:56<11:00,  3.02it/s]


KeyboardInterrupt: 

# 2. 12개월 변동성

In [10]:
# 최근 6개월(180일) 데이터를 조회할지 설정
days_ago = 180

# 오늘 날짜 계산
today = datetime.datetime.today().strftime('%Y%m%d')

# 시작 날짜 계산 (days_ago 일 전)
start_date = (datetime.datetime.today() - datetime.timedelta(days=days_ago)).strftime('%Y%m%d')

# KOSPI 전 종목 리스트 가져오기
all_tickers = list(df_tickers['Ticker'])

# 일일 변동성 지수 및 월별 수익률을 저장할 리스트
volatility_data = []

# 각 종목의 시세 데이터를 조회하고 변동성 및 월별 수익률 계산
for ticker in tqdm(all_tickers, desc="Processing..."):
    # 각 종목의 일별 시세 조회 (종가 기준)
    df = stock.get_market_ohlcv_by_date(start_date, today, ticker)

    # 시작 시점의 주가 (첫 번째 거래일의 종가)
    start_price = df['종가'].iloc[0]

    # # 마지막 시점의 주가 (가장 최근 거래일의 종가)
    current_price = df['종가'].iloc[-1]

    # 종가 기준 일일 수익률 계산
    df['daily_return'] = df['종가'].pct_change()

    # 일일 변동성 계산 (표준편차)
    daily_volatility = round(df['daily_return'].std() * 100, 2)  # 변동성을 %로 변환하고 소수점 둘째 자리까지

    # 일일 변동성을 6개월로 나누기 위한 계산
    df['month'] = df.index.to_period('M')  # 날짜를 월 단위로 변환
    monthly_volatility = df.groupby('month')['daily_return'].std() * 100  # 월별 변동성 계산
    monthly_volatility_list = [round(x, 2) for x in monthly_volatility.tolist()[-6:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (6개월보다 적은 데이터 처리)
    while len(monthly_volatility_list) < 6:
        monthly_volatility_list.insert(0, 0)

    # 수익률 계산 (시작 주가와 마지막 주가 기준)
    total_return = round(((current_price / start_price) - 1) * 100, 2)  # 수익률을 %로 계산하고 소수점 둘째 자리까지

    # 월별 수익률 계산 (6개월)
    monthly_data = df.groupby('month')['종가'].agg(['first', 'last'])

    # 월별 수익률 계산 (시가와 종가 차이로 계산)
    monthly_data['return'] = (monthly_data['last'] / monthly_data['first'] - 1) * 100

    # 6개월 수익률을 리스트에 저장 (가장 최근 달부터 과거로)
    monthly_return_list = [round(x, 2) for x in monthly_data['return'].tolist()[-6:]]  # 소수점 둘째 자리까지

    # 부족한 월은 0으로 채움 (6개월보다 적은 데이터 처리)
    while len(monthly_return_list) < 6:
        monthly_return_list.insert(0, 0)

    # 샤프 비율 계산 (수익률 / 변동성)
    monthly_sharpe_ratio_list = []
    for i in range(6):
        if monthly_volatility_list[i] == 0:  # 변동성이 0일 경우 NaN으로 처리
            monthly_sharpe_ratio_list.append(float('nan'))
        else:
            monthly_sharpe_ratio_list.append(round(monthly_return_list[i] / monthly_volatility_list[i], 2))

    # 변동성 정보 및 월별 수익률을 저장 (변동성 컬럼을 사이사이에 추가)
    volatility_data.append({
        'Ticker': ticker,
        #'Name': company_name,
        'Start Price': round(start_price, 2),  # 시작 주가 소수점 둘째 자리까지
        'Current Price': round(current_price, 2),  # 현재 주가 소수점 둘째 자리까지
        'Total Return (%)': total_return,  # 총 수익률 (%)
        'Month 1 Return (%)': monthly_return_list[-1],  # 가장 최근 월의 수익률
        'Month 2 Return (%)': monthly_return_list[-2],
        'Month 3 Return (%)': monthly_return_list[-3],
        'Month 4 Return (%)': monthly_return_list[-4],
        'Month 5 Return (%)': monthly_return_list[-5],
        'Month 6 Return (%)': monthly_return_list[-6],  # 6개월 전의 수익률
        'Month 1 Volatility (%)': monthly_volatility_list[-1],  # 가장 최근 월의 변동성
        'Month 2 Volatility (%)': monthly_volatility_list[-2],
        'Month 3 Volatility (%)': monthly_volatility_list[-3],
        'Month 4 Volatility (%)': monthly_volatility_list[-4],
        'Month 5 Volatility (%)': monthly_volatility_list[-5],
        'Month 6 Volatility (%)': monthly_volatility_list[-6],  # 6개월 전의 변동성
        'Month 1 Sharpe Ratio': monthly_sharpe_ratio_list[-1],  # 가장 최근 월의 샤프 비율
        'Month 2 Sharpe Ratio': monthly_sharpe_ratio_list[-2],
        'Month 3 Sharpe Ratio': monthly_sharpe_ratio_list[-3],
        'Month 4 Sharpe Ratio': monthly_sharpe_ratio_list[-4],
        'Month 5 Sharpe Ratio': monthly_sharpe_ratio_list[-5],
        'Month 6 Sharpe Ratio': monthly_sharpe_ratio_list[-6]  # 6개월 전의 샤프 비율
    })

# 변동성 데이터를 데이터프레임으로 변환
volatility_df = pd.DataFrame(volatility_data)

# 인덱스 초기화
volatility_df.reset_index(drop=True, inplace=True)

# DataFrame 출력
volatility_df.to_excel('volatility_df.xlsx', index=False)


Processing...: 100%|██████████| 50/50 [00:01<00:00, 37.84it/s]


# 3. Merge

In [12]:
merged_df = pd.merge(df_tickers, volatility_df, how='left', left_on='Ticker', right_on='Ticker')
merged_df['Chart_Link'] = merged_df.apply(lambda row: f'=HYPERLINK("https://finance.naver.com/item/fchart.naver?code={row["Ticker"]}", "{row["Ticker_name"]} 차트보기")', axis=1)
"""
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ticker                  50 non-null     object  
 1   Market                  50 non-null     object 
 2   Ticker_name             50 non-null     object 
 3   Avg_Volume_30D          50 non-null     float64 (거래량 30일 평균)
 4   Avg_Trading_Value       50 non-null     float64 (거래금액 30일 평균)
 5   PER                     49 non-null     float64
 6   PBR                     49 non-null     float32
 7   Dividend_Yield          49 non-null     float32
 8   Market_Cap              50 non-null     float64
 9   Start Price             50 non-null     int64  
 10  Current Price           50 non-null     int64  
 11  Total Return (%)        50 non-null     float64
 12  Month 1 Return (%)      50 non-null     float64
 13  Month 2 Return (%)      50 non-null     float64
 14  Month 3 Return (%)      50 non-null     float64
 15  Month 4 Return (%)      50 non-null     float64
 16  Month 5 Return (%)      50 non-null     float64
 17  Month 6 Return (%)      50 non-null     float64
 18  Month 1 Volatility (%)  50 non-null     float64
 19  Month 2 Volatility (%)  50 non-null     float64
 20  Month 3 Volatility (%)  50 non-null     float64
 21  Month 4 Volatility (%)  50 non-null     float64
 22  Month 5 Volatility (%)  50 non-null     float64
 23  Month 6 Volatility (%)  50 non-null     float64
 24  Month 1 Sharpe Ratio    50 non-null     float64
 25  Month 2 Sharpe Ratio    50 non-null     float64
 26  Month 3 Sharpe Ratio    50 non-null     float64
 27  Month 4 Sharpe Ratio    50 non-null     float64
 28  Month 5 Sharpe Ratio    50 non-null     float64
 29  Month 6 Sharpe Ratio    50 non-null     float64
 30  Chart_Link              50 non-null     object 
dtypes: float32(2), float64(23), int64(2), object(4)
memory usage: 11.8+ KB
"""
merged_df.to_excel('merged_df.xlsx',index=False)

In [13]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ticker                  50 non-null     object 
 1   Market                  50 non-null     object 
 2   Ticker_name             50 non-null     object 
 3   Avg_Volume_30D          50 non-null     float64
 4   Avg_Trading_Value       50 non-null     float64
 5   PER                     49 non-null     float64
 6   PBR                     49 non-null     float32
 7   Dividend_Yield          49 non-null     float32
 8   Market_Cap              50 non-null     float64
 9   Start Price             50 non-null     int64  
 10  Current Price           50 non-null     int64  
 11  Total Return (%)        50 non-null     float64
 12  Month 1 Return (%)      50 non-null     float64
 13  Month 2 Return (%)      50 non-null     float64
 14  Month 3 Return (%)      50 non-null     floa